In [1]:
import os
from dotenv import load_dotenv
from getpass import getpass

# Load environment variables from .env file
load_dotenv()

# Get API base URL and model with default values
OPENAI_MODEL = os.getenv("OPENAI_MODEL", "gpt-4o-mini")
OLLAMA_API_BASE = os.getenv("OLLAMA_API_BASE", "http://localhost:11434/v1")
OLLAMA_API_KEY = os.getenv("OLLAMA_API_KEY", "ollama")
OLLAMA_MODEL = os.getenv("OLLAMA_MODEL", "qwen2.5:7b")

# Set up OpenAI API configuration
# Try to get API key from environment variables
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
# If not found, ask user to input it
if not OPENAI_API_KEY:
    OPENAI_API_KEY = getpass("Enter your OpenAI API key:")

from agents import OpenAIChatCompletionsModel, AsyncOpenAI

gpt4om_model= OpenAIChatCompletionsModel(
    model="gpt-4o-mini",
    openai_client=AsyncOpenAI(api_key=OPENAI_API_KEY),
)

qwen_model= OpenAIChatCompletionsModel(
    model="qwen2.5:7b",
    openai_client=AsyncOpenAI(base_url=OLLAMA_API_BASE,api_key=OLLAMA_API_KEY),
)

import json
from agents import function_tool

def get_form_schema() -> list:
    """Get the schema definition of the form data"""
    return [
        {
            "name": "username",
            "type": "string",
            "required": True,
            "description": "User name, between 3 and 20 characters long",
            "value": None,
        },
        {
            "name": "age",
            "type": "number",
            "required": False,
            "description": "Age, please enter a positive integer",
            "value": None,
        },
        {
            "name": "email",
            "type": "string",
            "required": True,
            "description": "Email address to receive notifications",
            "value": None,
        }
    ]

def ensure_form_file_exists(file_path: str) -> bool:
    """
    Ensure the form data file exists, create it if it doesn't
    """
    try:
        if not os.path.exists(file_path):
            print(f"Form data file does not exist: {file_path}, creating new file")
            # Ensure directory exists
            os.makedirs(os.path.dirname(file_path), exist_ok=True)
            # Create empty form data
            empty_form_data = get_form_schema()
            # Write empty data to file
            with open(file_path, 'w', encoding='utf-8') as f:
                json.dump(empty_form_data, f, ensure_ascii=False, indent=2)
        return True
    except Exception as e:
        print(f"Error creating form data file: {e}")
        return False

@function_tool
def get_form_data_tool() -> str:
    """
    Read the form data filled by the user
    """
    file_path = "./form_data.json"
    try:
        if not ensure_form_file_exists(file_path):
            return get_form_schema()

        with open(file_path, 'r', encoding='utf-8') as f:
            form_data = json.load(f)
            return json.dumps(form_data, indent=2)
            # return form_data
    except json.JSONDecodeError:
        return(f"JSON format error: {file_path}")
    except Exception as e:
        return(f"Error reading form data: {e}")

@function_tool
def update_form_field_tool(field_name:str, field_value: str|int|float) -> str:
    """
    Update a field in the form data

    Args:
        field_name (str): Form field name
        field_value (str): Form field value
    
    Returns:
        Updated form data
    """
    file_path = "./form_data.json"
    try:
        # Ensure file exists
        if not ensure_form_file_exists(file_path):
            return
            
        with open(file_path, 'r+', encoding='utf-8') as f:
            form_data = json.load(f)
            # Find the field to update
            for field in form_data:
                if field['name'] == field_name:
                    # Convert value according to field type
                    if field['type'] == 'number':
                        try:
                            field['value'] = int(field_value)
                        except ValueError:
                            try:
                                field['value'] = float(field_value)
                            except ValueError:
                                return(f"Warning: Cannot convert '{field_value}' to number type, keeping original string")
                                # field['value'] = field_value
                    else:  # string or other types
                        field['value'] = field_value
                    break
            f.seek(0)
            json.dump(form_data, f, ensure_ascii=False, indent=2)
            f.truncate()
        return f"Form data update successful: {field_name} = {field_value}"
    except Exception as e:
        return(f"Error updating form data: {e}")

In [2]:
from agents import Agent

form_agent = Agent(
    name="Form Assistant",
    instructions="You are an assistant helping users fill out form data. Please query and update form data based on user input.",
    tools=[
        get_form_data_tool,
        update_form_field_tool
    ],
    model=qwen_model,
    # model=gpt4om_model,
)

In [4]:
from agents import Runner, RunResultStreaming
from openai.types.responses import (
    ResponseTextDeltaEvent,
    ResponseFunctionCallArgumentsDeltaEvent,
)

async def show_streaming_response(
    response: RunResultStreaming,
    verbose: bool = False,
) -> None:
    async for event in response.stream_events():
        # print(f'\n{event.type}')
        # print(event)
        if event.type == "raw_response_event": 
            if isinstance(event.data, ResponseFunctionCallArgumentsDeltaEvent) and verbose:
                # this is streamed parameters for our tool call
                print(event.data.delta, end="", flush=True)
            elif isinstance(event.data, ResponseTextDeltaEvent):
                # this is streamed final answer tokens
                print(event.data.delta, end="", flush=True)
        elif event.type == "agent_updated_stream_event" and verbose:
            # this tells us which agent is currently in use
            print(f"> Current Agent: {event.new_agent.name}")
        elif event.type == "run_item_stream_event" and verbose:
            # these are events containing info that we'd typically
            # stream out to a user or some downstream process
            if event.name == "tool_called":
                # this is the collection of our _full_ tool call after our tool
                # tokens have all been streamed
                print()
                print(f"> Tool Called, name: {event.item.raw_item.name}")
                print(f"> Tool Called, args: {event.item.raw_item.arguments}")
            elif event.name == "tool_output":
                # this is the response from our tool execution
                print(f"> Tool Output: {event.item.raw_item['output']}")
        elif verbose:
            print(f"Unknown event type: {event.type}", type(event).__name__)
            print(event)

async def handle_customer_query(query: str) -> None:
    response = Runner.run_streamed(form_agent,query)
    await show_streaming_response(response)

In [6]:
await handle_customer_query("Please show all the form data")

Here is the current form data:

- Username: new_username (3 to 20 characters required)
- Age: Not provided
- Email: abc@abc.com

In [7]:
await handle_customer_query("What fields are required? Help me list all the required fields and current values")

The required fields and their current values are as follows:

- **Username**: `new_username` (3-20 characters)
- **Email**: `abc@abc.com` (used for receiving notifications) 

Please note that the Age field is optional and currently has no value entered. If you need any further assistance or have changes to make, let me know!

In [8]:
await handle_customer_query("Set Age to 27.7777")

The age has been successfully updated to 27.7777 in the form data.

In [9]:
await handle_customer_query("what felds I didn't fill out?")

Based on the form data you've filled out, it seems that you haven't provided an age. The username and email fields have been filled correctly. Please provide your age so we can complete the form.